In [15]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

In [18]:
# Generate synthetic data
np.random.seed(42)
X = np.random.rand(100, 1) * 10  # Feature
y = 2 * X.squeeze() + np.random.randn(100) * 2  # Target with noise

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Add constant term for intercept
X_train_with_const = sm.add_constant(X_train_scaled)
X_test_with_const = sm.add_constant(X_test_scaled)

# Define quantiles
quantiles = [0.25, 0.5, 0.75]

# Train quantile regression models
models = {}
for q in quantiles:
    model = sm.QuantReg(y_train, X_train_with_const).fit(q=q)
    models[q] = model

# Make predictions on test set
predictions = {q: models[q].predict(X_test_with_const) for q in quantiles}

# Predict for new data
new_X = np.array([[5.5]])  # Example new input
new_X_scaled = scaler.transform(new_X)

# Fix: Ensure new input has the same structure as training data
new_X_with_const = sm.add_constant(new_X_scaled, has_constant='add')

# Predict new values
new_predictions = {q: models[q].predict(new_X_with_const)[0] for q in quantiles}

print("Predictions for new data (Feature=5.5):")
for q in quantiles:
    print(f"Quantile {q}: {new_predictions[q]:.4f}")

Predictions for new data (Feature=5.5):
Quantile 0.25: 9.3952
Quantile 0.5: 10.9006
Quantile 0.75: 11.8257
